In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import time

In [2]:
chartmetric_url = 'https://api.chartmetric.com'
post_request_url = f'{chartmetric_url}/api/token'

# Set environment variables from the .env in the local environment
load_dotenv(".env")

# Retrieve API key and store as Python variable
refresh_api_token = os.getenv("REFRESH_TOKEN")

In [3]:
refresh_api_token

In [4]:
post_request_url = chartmetric_url + '/api/token'

post_results = requests.post(post_request_url, data = {"refreshtoken":refresh_api_token}, json=True )
api_token = post_results.json()['token']
api_token

KeyError: 'token'

# Fresh Finds

In [ ]:
#Call  the fresh finds chartmetric URL, has to be a thursday in the month in order for API to return
date = '2024-06-27'
spotify_freshfind_url = f'https://api.chartmetric.com/api/charts/spotify/freshfind?date={date}'

#set headers t
headers = {"Authorization": f"Bearer {api_token}"}

#get results
spotify_freshfind_results = requests.get(spotify_freshfind_url, headers = headers)

In [ ]:
#display results JSON
spotify_freshfind_results.json()

In [ ]:
# Create a dataframe from the fresh finds data

freshfinds_df = pd.json_normalize(spotify_freshfind_results.json()['obj'])
freshfinds_df.head()

In [ ]:
# Identify Columns
freshfinds_df.columns

In [ ]:
# Identify data types
freshfinds_df.info

In [ ]:
# Reduce dataframe to only necessary columns
ff_df = freshfinds_df[['id',
                       'score',
                       'name', 
                       'isrc', 
                       'spotify_track_id', 
                       'spotify_album_id', 
                       'cm_track', 
                       'cm_artist',
                       'spotify_artist_names', 
                       'spotify_artist_ids',
                       'spotify_track_ids',
                       'spotify_album_ids',
                       'spotify_duration_ms',
                       'album_ids',
                       'album_names',
                       'track_genre',
                       'album_label',
                       'release_dates',
                       'composer_name',
                       'songwriter',
                       'artists',
                       'album',
                       'duration_ms',
                       'explicit',
                       'num_ff_playlists',
                       'fresh_finds_playlists',
                       'code2s'
                      ]]
ff_df

In [ ]:
# Create a list of dates to cycle through

date_range = [ 
         '2024-03-28',
         '2024-04-04', 
         '2024-04-11',
         '2024-04-18',
         '2024-04-25',
         '2024-05-02',
         '2024-05-09',
         '2024-05-16',
         '2024-05-23',
         '2024-05-30',
         '2024-06-06',
         '2024-06-13', 
         '2024-06-20']

# Initialize an empty list to store DataFrames
dfs = []

#loop through list and store API responses in a dataframe
for date in date_range:
    #call the API with that date
    spotify_freshfind_url = f'https://api.chartmetric.com/api/charts/spotify/freshfind?date={date}'
    #set headers t
    headers = {"Authorization": f"Bearer {api_token}"}

    #get results
    spotify_freshfind_results = requests.get(spotify_freshfind_url, headers = headers)

    # Check if the request was successful
    if spotify_freshfind_results.status_code == 200:
        # Parse the JSON response
        data = spotify_freshfind_results.json()
        
        # Normalize the JSON response to a DataFrame
        df = pd.json_normalize(data['obj'])  
        
        # Add a column for the date
        df['date'] = date
        
        # Append the DataFrame to the list
        dfs.append(df)
    else:
        print(f"Failed to retrieve data for date: {date}, Status code: {spotify_freshfind_results.status_code}")

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Output the final DataFrame
print(final_df)

In [ ]:
final_df

In [ ]:
final_df.shape

In [ ]:
# Aggregate track appearances over the specified dates
track_appearances = {}

for date in date_range:
    #call the API with that date
    spotify_freshfind_url = f'https://api.chartmetric.com/api/charts/spotify/freshfind?date={date}'
    #set headers t
    headers = {"Authorization": f"Bearer {api_token}"}

    #get results
    spotify_freshfind_results = requests.get(spotify_freshfind_url, headers = headers)
    
    for track in spotify_freshfind_results.json()['obj']:
        if track['spotify_track_id'] in track_appearances:
            track_appearances[track['spotify_track_id']] += 1
        else:
            track_appearances[track['spotify_track_id']] = 1

   # Wait for a specified amount of time before making the next request
    time.sleep(2)  # Wait for 2 seconds (adjust the duration as needed)
    
track_appearances

In [ ]:
#Calculate median 'Time_on_chart'
median_time_on_chart = pd.Series(track_appearances.values()).median()

median_time_on_chart

In [ ]:
# Function to determine if a track is a hit
def check_if_hit(track_id):
     appearances = track_appearances.get(track_id, 0)
     if appearances > median_time_on_chart:
         return "Big Hit"
     else:
         return "Low Hit"

# Example use of check_if_hit
for track in track_appearances:
    track_id_example = track
    hit_status = check_if_hit(track_id_example)
    print(f"The track with ID {track_id_example} is a {hit_status}.")

In [ ]:
# Specify the path and name of the CSV file
output_csv_path = 'ListOfSpotifyTracksThatAreHits.csv'

# Save the DataFrame to a CSV file
final_df.to_csv(output_csv_path, index=False)  # Set index=False to exclude row indices in the CSV file

print(f"DataFrame is saved to {output_csv_path}")

# EDA -Disregard

In [ ]:
#get spotify charts

spotify_artist_charts_url = f"https://api.chartmetric.com/api/charts/spotify/artists?latest=true&interval=monthly&type=popularity"

headers = {"Authorization": f"Bearer {api_token}"}

spotify_artist_charts_results = requests.get(spotify_artist_charts_url, headers = headers)

In [ ]:
spotify_artist_charts_results.json()

In [ ]:
artist_objects = spotify_artist_charts_results.json()['obj']['data']


#grab the name, sp_popularity, 'sp_monthly_listeners', 'spotify_artist_ids', 'monthly_popularity', 'rank', 'streak'
artist_popularity_keys = ['name', 'sp_popularity', 'sp_monthly_listeners', 'spotify_artist_ids', 'monthly_popularity', 'rank', 'streak']

#create an empyt list to store df data
artist_popularity_df = []

#iterate through artist objects to get only relevant data, assign keys and values and append to list
for artist in artist_objects:
    artist_info = {key: artist.get(key) for key in artist_popularity_keys}
    extracted_data.append(artist_info)
#create the new artist popularity df 
artist_popularity_df = pd.DataFrame(extracted_data)
        

In [ ]:
#print top ranked artists for latest

artist_popularity_df.head()

In [ ]:
#pull top spotify charts

spotify_top_charts_url = 'https://api.chartmetric.com/api/charts/spotify?country_code=GLOBAL&type=plays&interval=daily&latest=true'

headers = {"Authorization": f"Bearer {api_token}"}

spotify_charts_results = requests.get(spotify_top_charts_url, headers = headers)


In [ ]:
spotify_charts_results.json()

In [ ]:
# pull tik tok charts
tik_tok_charts_url = 'https://api.chartmetric.com/api/charts/tiktok/top-tracks-stats'

